In [1]:
import pandas as pd
import collections
from util import gera_id_gestao
pd.set_option('display.max_columns', None)

In [2]:
CRIMPAR_MAX_VARIACAO = 30 # valor nos 90% maiores
CRIMPAR_MIN_VARIACAO = -13 # valor nos 10% menores
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
LISTA_ANO_ANTERIOR = ['2013', '2014', '2015', '2016', '2017']

NOME_IDESP_AI = 'dados\idesp/IDESP_Escolas_2007_2018_AI.CSV'
NOME_IDESP_AF = 'dados\idesp/IDESP_Escolas_2007_2018_AF.CSV'
NOME_IDESP_EM = 'dados\idesp/IDESP_Escolas_2007_2018_EM.CSV'

# ISSO AQUI NAO DEVERIA ENTRAR NA CONFECCAO DO DATA MART
# FAZ PARTE DAS ANALISES
#idesp_group.variacao = idesp_group.variacao.apply(lambda x: CRIMPAR_MAX_VARIACAO if x > CRIMPAR_MAX_VARIACAO else x)
#idesp_group.variacao = idesp_group.variacao.apply(lambda x: CRIMPAR_MIN_VARIACAO if x < CRIMPAR_MIN_VARIACAO else x)
 

In [3]:
def etl():
    df_idesp = extracao()
    df_idesp = limpeza(df_idesp)
    df_idesp = agrupamento_id_gestao(df_idesp)
    return df_idesp

In [4]:
def enulo(valor):
    '''TODO - fortalecer essa conferencia de nulo'''
    return str(valor) == 'nan'   

def atual_enulo(dados):
    '''Verifca se a prova idesp foi aplicada no ano da gestao
    ENTRADA
    * dados: dados da tabela IDESP, limpos e agrupados pelo id_gestao
    SAIDA
    * atual_nulo: 0 se todas as provas foram aplicadas, 1 se a aplicacao foi parcial
                  2 se nenhuma das provas foi aplicada.'''
    soma_atual = dados.atual_nulo.sum()
    if soma_atual == 0:
        return 0
    elif soma_atual == len(dados):
        return 2
    else:
        return 1

In [5]:
def extracao():
    df_idesp_anosiniciais = pd.read_csv(NOME_IDESP_AI, sep=',')
    df_idesp_anosfinais = pd.read_csv(NOME_IDESP_AF, sep=',')
    df_idesp_ensinomedio = pd.read_csv(NOME_IDESP_EM, sep=',')
    df_idesp_anosfinais = df_idesp_anosfinais.dropna(thresh=2)  # DESCARTE DE 1 LINHA VAZIA
    df_idesp = df_idesp_anosiniciais.append(df_idesp_anosfinais).append(df_idesp_ensinomedio)
    return df_idesp

In [32]:
def limpeza(df_idesp):
    '''Na tabela df_idesp os anos de um mesmo nivel de ensino
    e escola ficam na mesma linha. Precisamos que cada resultado
    do IDESP esteja em uma linha separada. Adicionalmente, e necessario
    descartar anos onde a prova nao foi aplicada.'''
    idesp = {}
    idesp['id_gestao'] = []
    idesp['id_escola'] = []
    idesp['ano_gestao'] = []
    idesp['nivel_ensino'] = []
    idesp['nota_ano'] = []
    idesp['nota_anterior'] = []
    idesp['anterior_nulo'] = []
    idesp['atual_nulo'] = []

    for _, row in df_idesp.iterrows():
        for ano, ano_anterior in zip(LISTA_ANOS, LISTA_ANO_ANTERIOR):
            atual_nulo = 0
            idesp_atual = row.loc[ano]
            if enulo(idesp_atual):
                atual_nulo = 1
                idesp_atual = 0
            idesp_anterior = row.loc[ano_anterior]
            anterior_nulo = 0
            if enulo(idesp_anterior):
                anterior_nulo = 1
                idesp_anterior = idesp_atual

            idesp['id_gestao'].append(gera_id_gestao(row.loc['CODIGO CIE'], ano))
            idesp['id_escola'].append(int(row.loc['CODIGO CIE']))
            idesp['ano_gestao'].append(int(ano))
            idesp['nivel_ensino'].append(row.loc['NIVEL ENSINO'])
            idesp['nota_ano'].append(idesp_atual)
            idesp['nota_anterior'].append(idesp_anterior)
            idesp['anterior_nulo'].append(anterior_nulo)
            idesp['atual_nulo'].append(atual_nulo)
    
    return pd.DataFrame(data=idesp)

In [63]:
def agrupamento_id_gestao(df_idesp):
    idesp_group = {}
    idesp_group['id_gestao'] = []
    idesp_group['id_escola'] = []
    idesp_group['ano_gestao'] = []
    idesp_group['nota_ano'] = []
    idesp_group['variacao'] = []
    idesp_group['anterior_nulo'] = []
    idesp_group['atual_nulo'] = []

    for nome, dados in df_idesp.groupby('id_gestao'):
        atual_nulo = atual_enulo(dados)
        if atual_nulo == 2:
            nota_ano = None
            nota_anterior = None
            variacao = None
        else:
            if atual_nulo == 1:
                dados = dados[dados.atual_nulo == 0]
            nota_ano = dados.nota_ano.mean()
            nota_anterior = dados.nota_anterior.mean()
            variacao = 100 * (nota_ano - nota_anterior) / nota_anterior
        
        idesp_group['id_gestao'].append(nome)
        idesp_group['id_escola'].append(dados.id_escola.iloc[0])
        idesp_group['ano_gestao'].append(int(dados.ano_gestao.iloc[0]))
        idesp_group['nota_ano'].append(nota_ano)
        idesp_group['variacao'].append(variacao)
        idesp_group['anterior_nulo'].append(int(dados.anterior_nulo.sum() > 0))
        idesp_group['atual_nulo'].append(atual_nulo)
    
    idesp_group = pd.DataFrame(data=idesp_group)
    return idesp_group
    


### CHECANDO SE O NOME DAS COLUNAS E IGUAL

In [36]:
df_idesp_anosiniciais = pd.read_csv('dados\IDESP_Escolas_2007_2018_AI.CSV',sep=',')
df_idesp_anosfinais = pd.read_csv('dados\IDESP_Escolas_2007_2018_AF.CSV',sep=',')
df_idesp_ensinomedio = pd.read_csv('dados\IDESP_Escolas_2007_2018_EM.CSV',sep=',')
df_idesp_anosfinais = df_idesp_anosfinais.dropna(thresh=2)

In [37]:
df_idesp_anosiniciais.columns == df_idesp_anosfinais.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [38]:
df_idesp_anosiniciais.columns == df_idesp_ensinomedio.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [39]:
df_idesp = df_idesp_anosiniciais.append(df_idesp_anosfinais).append(df_idesp_ensinomedio)

In [40]:
df_idesp.loc[:,'CODIGO CIE'] = df_idesp.loc[:,'CODIGO CIE'].astype(int)

### Linha vazia encontrada e descartada

In [13]:
collections.Counter(df_idesp_anosfinais.loc[:,'NIVEL ENSINO'])

Counter({'ANOS FINAIS': 4052})

In [14]:
df_idesp_anosfinais[df_idesp_anosfinais.loc[:,'NIVEL ENSINO'].isnull()]

,CODIGO CIE,CODIGO INEP,DIRETORIA,ESCOLA,MUNICIPIO,NIVEL ENSINO,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


# ESCOLHA DO INDICADOR

In [41]:
df_idesp = limpeza(df_idesp)

In [64]:
idesp = agrupamento_id_gestao(df_idesp)

In [71]:
idesp.head(3)

,id_gestao,id_escola,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1
1,esc-10005-ano-2015,10005,2015,3.78,8.620690,0,1
2,esc-10005-ano-2016,10005,2016,3.26,-13.756614,0,1


In [59]:
    idesp_group = {}
    idesp_group['id_gestao'] = []
    idesp_group['id_escola'] = []
    idesp_group['ano_gestao'] = []
    idesp_group['nota_ano'] = []
    idesp_group['variacao'] = []
    idesp_group['anterior_nulo'] = []
    idesp_group['atual_nulo'] = []

    for nome, dados in df_idesp.groupby('id_gestao'):
        atual_nulo = atual_enulo(dados)
        if atual_nulo == 2:
            nota_ano = None
            nota_anterior = None
            variacao = None
        else:
            if atual_nulo == 1:
                dados = dados[dados.atual_nulo == 0]
            nota_ano = dados.nota_ano.mean()
            nota_anterior = dados.nota_anterior.mean()
            variacao = 100 * (nota_ano - nota_anterior) / nota_anterior
        
        idesp_group['id_gestao'].append(nome)
        idesp_group['id_escola'].append(dados.id_escola.iloc[0])
        idesp_group['ano_gestao'].append(int(dados.ano_gestao.iloc[0]))
        idesp_group['nota_ano'].append(nota_ano)
        idesp_group['variacao'].append(variacao)
        idesp_group['anterior_nulo'].append(int(dados.anterior_nulo.sum() > 0))
        idesp_group['atual_nulo'].append(atual_nulo)
    
    idesp_group = pd.DataFrame(data=idesp_group)
   


In [60]:
idesp_group.head(3)

,id_gestao,id_escola,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1
1,esc-10005-ano-2015,10005,2015,3.78,8.620690,0,1
2,esc-10005-ano-2016,10005,2016,3.26,-13.756614,0,1


In [61]:
collections.Counter(idesp_group.anterior_nulo)

Counter({0: 24588, 1: 2462})

In [62]:
collections.Counter(idesp_group.atual_nulo)

Counter({1: 4149, 0: 20962, 2: 1939})

In [58]:
1939/(20962+4149+1939)

0.07168207024029574

In [50]:
idesp_group.variacao.isnull().sum()

1693

In [25]:
725/24386  # DESTA ANALISE, HOUVERAM APENAS 3% DE VEZES EM QUE A PROVA ANTERIOR NAO FOI APLICADA

0.02973017305011072

In [1]:
import etl_idesp

In [2]:
idesp = etl_idesp.etl()